In [1]:
import re
from operator import itemgetter

import numpy as np
from obspy import Stream, Trace, UTCDateTime
import pyasdf

# infile = '/home/1/dev/proj/dugseis/doku/GMUG/20W20190701181032860.dat'
infile = '/home/1/dev/proj/dugseis/doku/GMUG/s-wave/Data/AGW20150325001449390'
dest_dir = '/home/1/dev/proj/dugseis/devel/processing/plotting/gmug/'
# data = np.fromfile(f'{infile}.dat', dtype='int16')
# traces = np.split(data, 32)

In [2]:
# read txt file
with open(f'{infile}.txt', 'r', encoding='iso-8859-15') as fh:
    lines = re.sub(r'\n+$', '', fh.read()).split('\n')

times = lines[18:]
times = [[int(el) for el in x] for x in [t.strip().split() for t in times]]
times = [UTCDateTime(t[5], t[6], t[7], t[1], t[2], t[3], t[4] * 1000) for t in times]
times

[2015-03-25T00:14:49.400000Z,
 2015-03-25T00:16:33.247000Z,
 2015-03-25T00:25:06.248000Z,
 2015-03-25T00:43:14.494000Z,
 2015-03-25T00:51:31.803000Z,
 2015-03-25T00:58:22.551000Z,
 2015-03-25T01:06:01.711000Z,
 2015-03-25T01:06:56.649000Z]

In [3]:
lines = itemgetter(*[i for i in range(4, 15) if i not in (12, 13)])(lines)
keys = ('delta', 'records_count', 'samples_count', 'channels_count', 'pre_trigger_precent', 
        'ad_resolution', 'input_range', 'trigger_level', 'digit_multiplier')
params = dict(zip(keys, [l.strip() for l in lines]))

# type conversions
params['delta'] = float(params['delta'])

# to integer
for key in ('delta', 'records_count', 'samples_count', 'channels_count', 'pre_trigger_precent', 
        'ad_resolution'):
    params[key] = int(params[key])

# to list of integers
for key in ('input_range', 'trigger_level', 'digit_multiplier'):
    params[key] = [int(x) for x in params[key].split()]

params['sampling_rate'] = 1 / (params['delta'] * 1e-6)

params

{'delta': 2,
 'records_count': 8,
 'samples_count': 16384,
 'channels_count': 16,
 'pre_trigger_precent': 15,
 'ad_resolution': 16,
 'input_range': [10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000,
  10000],
 'trigger_level': [8,
  8,
  8,
  1000,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  1000,
  1000,
  1000,
  1000],
 'digit_multiplier': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 'sampling_rate': 500000.0}

In [4]:
# read dat file
src_data = np.fromfile(f'{infile}.dat', dtype='int16')
print(len(src_data))
events = np.split(src_data, len(times))
# print(len(src_data))
# print(len(events[0]))

# len(events[0]) / 16

# tr = obspy.Trace(data=np.array([0., 0., 0., 0., 1., 2., 1., 0., 0.]))


# for evt_idx in len(times):
# for evt_idx in [0]:
#     stream = Stream()
#     channels = np.split(events[evt_idx], params['channels_count'])
#     for ch_idx in range(0, params['channels_count']):
#         data = channels[ch_idx]
#         stats = {
#             'network'      : 'IK',
#             'station'      : str(ch_idx + 1).zfill(3),
#             'location'     : '00',
#             'channel'      : '001',
#             'delta'        : params['delta'],
#             'sampling_rate': params['sampling_rate'],
#             'starttime'    : times[evt_idx],
#         }
#         stream += Trace(data=data, header=stats)

#         filename = times[3].isoformat().replace(':', '-').replace('.', '')[:-2] + 'Z_gmug-v4.h5'
#     file_handle = pyasdf.ASDFDataSet(dest_dir + filename, compression=None)
# #     file_handle.add_waveforms(stream, tag='raw_recording')
#     file_handle.append_waveforms(stream, tag='raw_recording')

print('Done')

2097152
Done


In [5]:
stream = Stream()
evt_idx = 0
channels = np.split(events[evt_idx], params['channels_count'])

for ch_idx in range(0, params['channels_count']):
    data = channels[ch_idx]
    stats = {
        'network'      : 'IK',
        'station'      : str(ch_idx + 1).zfill(3),
        'location'     : '00',
        'channel'      : '001',
#         'npts'         : 10,
        'delta'        : params['delta'],
        'sampling_rate': params['sampling_rate'],
        'starttime'    : times[evt_idx],
    }
    stream += Trace(data=data, header=stats)

filename = times[3].isoformat().replace(':', '-').replace('.', '')[:-2] + 'Z_gmug-xyz.h5'
file_handle = pyasdf.ASDFDataSet(dest_dir + filename, compression=None)
file_handle.add_waveforms(stream, tag='raw_recording')

In [6]:
load_files = ['2015-03-25T00-14-494000Z_gmug-v-mx-01.h5']
t_str = re.sub('Z.*', '', load_files[0])
li = re.split('[^0-9]', t_str)
li = li[:5] + [li[-1][0:2], li[-1][2:].ljust(6, '0')]
UTCDateTime(*[int(l) for l in li])

2015-03-25T00:14:49.400000Z